In [2]:
!pip install -r ./requirements.txts 

     ---------------------------------------- 0.0/48.9 kB ? eta -:--:--
     -------- ------------------------------- 10.2/48.9 kB ? eta -:--:--
     -------------------------------------- 48.9/48.9 kB 623.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
     -- ------------------------------------- 0.3/4.7 MB 6.5 MB/s eta 0:00:01
     ------- -------------------------------- 0.9/4.7 MB 9.2 MB/s eta 0:00:01
     -------------- ------------------------- 1.7/4.7 MB 12.0 MB/s eta 0:00:01
     ----------------------- ---------------- 2.8/4.7 MB 16.4 MB/s eta 0:00:01
     ----------------------------------- ---- 4.2/4.7 MB 19.3 MB/s eta 0:00:01
     ---------------------------------------- 4.7/4.7 MB 17.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: still running...
  Getting requirements to b

  error: subprocess-exited-with-error
  
  Preparing metadata (pyproject.toml) did not run successfully.
  exit code: 1
  
  [199 lines of output]
  setup.py:66: RuntimeWarning: NumPy 1.20.3 may not yet support Python 3.11.
    warnings.warn(
  Running from numpy source directory.
  setup.py:485: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
    run_build = parse_setuppy_commands()
  Processing numpy/random\_bounded_integers.pxd.in
  Processing numpy/random\bit_generator.pyx
  Processing numpy/random\mtrand.pyx
  Processing numpy/random\_bounded_integers.pyx.in
  Processing numpy/random\_common.pyx
  Processing numpy/random\_generator.pyx
  Processing numpy/random\_mt19937.pyx
  Processing numpy/random\_pcg64.pyx
  Processing numpy/random\_philox.pyx
  Processing numpy/random\_sfc64.pyx
  Cythonizing sources
  blas_opt_info:
  blas_mkl_info:
  No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying fro

### 투자 성과 지표
* 단일 종목 일간 수익률
* 단일 종목 누적 수익률
* CAGR
* MDD

In [1]:
import pandas as pd
import numpy as np

## 단일동목 일간 수익률
* 일간 수익률 = <span style="color:#008000">__당기(오늘) 가격 / 전기(어제) 가격__</span> 으로 계산 
* shift 메서드를 이용
* `shift(n)`은 n만큼 아래로 이동시키는 기능

In [11]:
data = {'close': [100, 95, 101, 103, 90, 95, 110, 98, 101]} # 9일 간 가격 데이터, n",
index = pd.to_datetime(["2022-05-17", "2022-05-18" , "2022-05-19", "2022-05-20", "2022-05-23", "2022-05-24", "2022-05-25", "2022-05-26", "2022-05-27"])
df = pd.DataFrame(data=data, index=index) # 데이터 프레임 생성n,
df

,close
2022-05-17,100
2022-05-18,95
2022-05-19,101
2022-05-20,103
2022-05-23,90
2022-05-24,95
2022-05-25,110
2022-05-26,98
2022-05-27,101


In [5]:
df.shift(1)

,close
2022-05-17,NaN
2022-05-18,100.0
2022-05-19,95.0
2022-05-20,101.0
2022-05-23,103.0
2022-05-24,90.0
2022-05-25,95.0
2022-05-26,110.0
2022-05-27,98.0


In [8]:
dayReturn = df / df.shift(1)
dayReturn = dayReturn.fillna(1)  # null 값은 1로 교체
dayReturn

,close
2022-05-17,1.000000
2022-05-18,0.950000
2022-05-19,1.063158
2022-05-20,1.019802
2022-05-23,0.873786
2022-05-24,1.055556
2022-05-25,1.157895
2022-05-26,0.890909
2022-05-27,1.030612


## 단일종목 누적 수익률
* 주가의 움직임은 복리
* 일간 수익률을 누적(계속) 곱해주면 누적 수익률을 구할 수 있다
* `cumprod()` 사용

In [9]:
cumulativeReturn = dayReturn.cumprod()
cumulativeReturn

,close
2022-05-17,1.00
2022-05-18,0.95
2022-05-19,1.01
2022-05-20,1.03
2022-05-23,0.90
2022-05-24,0.95
2022-05-25,1.10
2022-05-26,0.98
2022-05-27,1.01


* 여기서 중요한 점은, 누적 수익률은 <span style="color:#008000">__자산의 흐름__</span>과 같다
* 내가 1원을 투자했다면, 1원이 주가변동에 따라 최종 얼마가 되었는지 알려주는 지표 

### CAGR (compound annual growth rate)

    * 투자 수익률를 계산할 떄는 복리 개념이기 때문에, 흔히 알고 있는 산술평균이 아니라 기하평균으로 계산해야 합니다.
        - 산술평균 = 합의 평균
        - 기하평균 = 곱의 평균 
     
    * 1년차 수익률 5%
    * 2년차 수익률 10% ,
    * 2년 투자 기간동안의 연 평균 수익률 k라고 했을 때.. 
     
     $$k =  \sqrt{0.05 * 0.1} $$
 
* **하지만 우리는 앞으로 일간 데이터를 사용할 것이기 때문에,**
* **일간 데이터를 활용하여 연간 수익률을 계산하기 위해서는** `(일간 수익률 갯수) / (영업일)` 역수 제곱으로 연산합니다.
-- 일반적으로 금융공학에서는 1년의 총 영업일을 252일로 간주합니다.

In [12]:
len(df)

9

In [15]:
# cumulativeReturn.iloc[-1] 9일간 최종적으로 얻은 수익률 1.01
cagr = cumulativeReturn.iloc[-1] ** (252 / len(df))
cagr

close    1.321291
Name: 2022-05-27 00:00:00, dtype: float64

### Draw Down (하락폭) && Max Draw Down (최대하락폭)

* 최대 낙폭지수, <span style="color:#008000"> 투자 기간 동안 고점부터 떨어진 낙폭 최댓값을 의미</span>  
* 최종 수익률만 보고 판단하기 쉬운데, 사실 투자 기간동안 50%가 빠지면 심리적 타격도 있기 때문에 손절하고 투자를 중단할 수도 있습니다. 그렇기 때문에 최대 낙폭은 가능한 낮을수록 좋습니다.

- 하락폭(DD) = $ \frac{최고점 - 현재}{최고점} $  
- 최대하락폭(MDD) = max ($ \frac{최고점 - 현재}{최고점} $)

`cummax()` 메서드를 사용하면 데이터를 위에서부터 아래로 조회하며 최대값을 갱신

In [16]:
cumulativeReturn

,close
2022-05-17,1.00
2022-05-18,0.95
2022-05-19,1.01
2022-05-20,1.03
2022-05-23,0.90
2022-05-24,0.95
2022-05-25,1.10
2022-05-26,0.98
2022-05-27,1.01


In [18]:
# 누적 최댓값
cumulativeReturn.cummax() 

,close
2022-05-17,1.00
2022-05-18,1.00
2022-05-19,1.01
2022-05-20,1.03
2022-05-23,1.03
2022-05-24,1.03
2022-05-25,1.10
2022-05-26,1.10
2022-05-27,1.10


### MDD 계산

In [20]:
dd = (cumulativeReturn.cummax() - cumulativeReturn) / cumulativeReturn.cummax() * 100
dd

,close
2022-05-17,0.000000
2022-05-18,5.000000
2022-05-19,0.000000
2022-05-20,0.000000
2022-05-23,12.621359
2022-05-24,7.766990
2022-05-25,0.000000
2022-05-26,10.909091
2022-05-27,8.181818


In [21]:
mdd = dd.max()
mdd

close    12.621359
dtype: float64